In [1]:
# necessary packages #

#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2
#using NearestNeighbors # Can't use KDtree in julia for posterior prediction

In [2]:
include("../../../util2.j")

colnorm (generic function with 1 method)

In [3]:
# packages for diagnostic test #

using StatsPlots
using MCMCChains
using PrettyTables

In [10]:
@load "../data/sim1data.jld";
@load "../results/ω_incp_pos_mean_conj_NNGP.jld"

1-element Array{Symbol,1}:
 :ω_incp_pos_mean

In [11]:
# plot the pictures of recovered latent process
using RCall
@rput ω_incp_obs
@rput coords_ord
@rput S
@rput ω_incp_pos_mean
R"""
library(MBA)
library(classInt)
library(RColorBrewer)
library(sp)
library(coda)
library(spBayes)
library(fields)

h <- 12
surf.raw1 <- mba.surf(cbind(coords_ord[S, ], ω_incp_obs[S, 1]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.raw2 <- mba.surf(cbind(coords_ord[S, ], ω_incp_obs[S, 2]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.fit1 <- mba.surf(cbind(coords_ord[S, ], ω_incp_pos_mean[, 1]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.fit2 <- mba.surf(cbind(coords_ord[S, ], ω_incp_pos_mean[, 2]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est


surf.brks <- classIntervals(c(surf.raw1[["z"]], surf.raw2[["z"]], surf.fit1[["z"]], 
                surf.fit2[["z"]]), 500, 'pretty')$brks
col.pal <- colorRampPalette(brewer.pal(11,'RdBu')[11:1])
xlim <- c(0, 1.13)

zlim <- range(c(surf.raw1[["z"]], surf.raw2[["z"]], surf.fit1[["z"]], surf.fit2[["z"]]))

# size for the mapping of w               
width <- 360
height <- 360
pointsize <- 16

jpeg(paste("../../pics/sim1_map-w1-true_conj.jpg", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.raw1)
plot(coords_ord[S, ], typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

jpeg(paste("../../pics/sim1_map-w2-true_conj.jpg", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.raw2)
plot(coords_ord[S, ], typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

jpeg(paste("../../pics/sim1_map-w1-fit_conj.jpg", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.fit1)
plot(coords_ord[S, ], typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

jpeg(paste("../../pics/sim1_map-w2-fit_conj.jpg", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.fit2)
plot(coords_ord[S, ], typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

"""

RObject{IntSxp}
null device 
          1 
